In [ ]:
"""

This script loads the Speech Commands dataset, builds the SC09 dataset (discards all samples except for
the utterances from 0 to 9), preprocesses the data and saves it to a designated folder as TFRecords. 

Note, in order to not discard the 0-9 utterances they must be in folders named 'zero', 'one', 'two', etc.
within the Speech Command dataset folder.

"""

In [ ]:
import argparse
import os
import sys
import tensorflow as tf
from tqdm import tqdm



__author__ = "Matthaios Stylianidis"


In [ ]:
# TODO: Move these functions to utils.py and import them
from pysndfile import sndio
import numpy as np

def load_audio(file_path):
    """Loads audio data from wav file using pysndfile.

    Args:
        file_path: Path to a wav file.

    Returns:
        A tuple containing the samples and the sampling rate of the
        wav file, in this order.
    """
    data = sndio.read(file_path)
    sampling_rate = data[1]
    samples = np.array(data[0], dtype=np.float32)
    return samples, sampling_rate

In [ ]:
def get_arguments():
    """Parse all the arguments provided from the CLI.
    
    Returns:
      A list of parsed arguments.
    """
    parser = argparse.ArgumentParser(description="Dataset preprocessing.")
    parser.add_argument("img_path", type=str,
                        help="Path to the RGB image file.")
    parser.add_argument("img_path", type=str,
                        help="Path to the RGB image file.")
    return parser.parse_args()


In [ ]:
DEFAULT_DATASET_PATH = './SpeechCommands/' # Default Speech Commands dataset path 
DEFAULT_PREPROCESSED_DATASET_PATH = './SC09_Preprocessed/' # Default path to save preprocessed data
DEFAULT_FIXED_SIGNAL_SIZE = 16384
RANDOM_PADDING = False

# Folders with 0-9 classes in speech commands
class_folders = [ 'zero', 'one', 'two', 'three', 'four', 'five', 'six', 'seven', 'eight','nine']

In [ ]:
#args = get_arguments()
#print(args.accumulate(args.integers))

In [ ]:
# Create directory for saving the pre-processed data if it does not exist
if not os.path.exists(DEFAULT_PREPROCESSED_DATASET_PATH):
    os.makedirs(DEFAULT_PREPROCESSED_DATASET_PATH)
    print("Created " + DEFAULT_PREPROCESSED_DATASET_PATH + " directory.")

In [ ]:
# Build the tensorflow preprocessing graph
audio_file_name = tf.placeholder(tf.string, [])

# Create tf operation that loads audio and returns samples and sampling rate
audio_samples, audio_sampling_freq = tf.py_func(load_audio, [audio_file_name], [tf.float32, tf.int64], \
                                               name='Data_loading') 

# Constant for desired signal size after cropping or padding
fixed_signal_size = tf.constant(DEFAULT_FIXED_SIGNAL_SIZE)

# Preprocess audio
if RANDOM_PADDING:
    # Crop sides if larger than fixed length, insert random padding otherwise
    #preprocessed_samples = tf.py_func(crop_or_random_pad, [audio_samples, fixed_signal_size], tf.float32)
    pass
else:
    # Crop sides if larger than fixed length, pad the sides of the signal symmetrically if not large enough
    tensor_height = tf.constant(1)
    reshaped_samples = tf.reshape(audio_samples, [1, -1])
    preprocessed_samples = tf.image.resize_image_with_crop_or_pad(reshaped_samples, tensor_height, fixed_signal_size)
    pass

sess = tf.Session()

In [ ]:
# For each folder in the dataset folder
sample_count = 0
for folder_name in os.listdir(DEFAULT_DATASET_PATH):
    # Ommit the folder if its name does not correspond to 0-9 digits.
    if folder_name not in class_folders:
        continue
    
    
    new_folder_path = DEFAULT_PREPROCESSED_DATASET_PATH + folder_name + '/'
    # Create folder with the same name in saving directory
    if not os.path.exists(new_folder_path):
        os.makedirs(new_folder_path)
        print("Created " + new_folder_path + " directory.")
    
    # Create TFRecord writer
    writer = tf.python_io.TFRecordWriter(new_folder_path + 'train.tfrecords')
    
    folder_path = DEFAULT_DATASET_PATH + folder_name + "/"
    print("Preprocessing samples from folder " + folder_path + ".")
    folder_file_count = 0
    for wav_file_name in tqdm(os.listdir(folder_path)):
        # Make ndarray with wav file path
        load_file_path = np.array(folder_path + wav_file_name)
        # Load and process the wav file using tensorflow
        preprocessed_data = sess.run(preprocessed_samples, feed_dict = {audio_file_name: load_file_path})
        print(preprocessed_data.shape)
        # Create a feature 
        feature = {'train/signal': _bytes_feature(tf.compat.as_bytes(preprocessed_data.tostring()))}
        # Create example protocol buffer
        exmple_protocol_buff = tf.train.Example(features=tf.train.Features(feature = feature))
        # Serialize to string and write to file
        writer.write(exmple_protocol_buff.SerializeToString())
        # Increase count of current class files by 1
        folder_file_count += 1
    
    # Close TFRecord writer
    writer.close()
    sys.stdout.flush()
    
    # Increase count of samples processed
    sample_count += folder_file_count
    
    print(str(folder_file_count) + " samples from folder " + folder_path + \
          " have been preprocessed and saved in " + \
          DEFAULT_PREPROCESSED_DATASET_PATH + folder_name + ".\n")
    
print("Loaded " + str(sample_count) + " wav files.")